Dataset : https://www.kaggle.com/lava18/google-play-store-apps

問題定義：哪個App下載數量會大於一百萬？

屬性分別有評分、評論數以及價錢

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz
import pydotplus
import os


In [2]:
df = pd.read_csv('./googleplaystore.csv')
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [3]:
df.isnull().sum()

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [4]:
Rating = df['Rating']
avgrate = Rating.sum(axis = 0, skipna = True) /len(Rating)
df['Rating'].fillna(avgrate, inplace = True)

In [5]:
size = len(df['Installs'])
size
Installs = df.pop('Installs')
Price = df.pop('Price')

In [6]:
for i in range(size):
    if Installs[i].endswith('+') != True:
        df = df.drop(i,axis = 0)
        Installs = Installs.drop(i,axis = 0)
        Price = Price.drop(i,axis = 0)
        size -= 1
    else:
        Installs[i] = Installs[i].replace('+','')
        Installs[i] = Installs[i].replace(',','')
        Installs[i] = int(Installs[i])
        if Price[i].isdigit() != True:
            Price[i] = Price[i].replace('$','')
            Price[i] = float(Price[i])
        else:
            Price[i] = float(Price[i])


In [7]:
df['Installs'] = Installs
df['Price'] = Price
df.sort_values(by=['Installs'],ascending=False)


,App,Category,Rating,Reviews,Size,Type,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Installs,Price
3665,YouTube,VIDEO_PLAYERS,4.30000,25655305,Varies with device,Free,Teen,Video Players & Editors,"August 2, 2018",Varies with device,Varies with device,1000000000,0
4104,Messenger – Text and Video Chat for Free,COMMUNICATION,4.00000,56642847,Varies with device,Free,Everyone,Communication,"August 1, 2018",Varies with device,Varies with device,1000000000,0
3896,Subway Surfers,GAME,4.50000,27711703,76M,Free,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up,1000000000,0
3996,Google Chrome: Fast & Secure,COMMUNICATION,4.30000,9642112,Varies with device,Free,Everyone,Communication,"August 1, 2018",Varies with device,Varies with device,1000000000,0
3909,Instagram,SOCIAL,4.50000,66509917,Varies with device,Free,Teen,Social,"July 31, 2018",Varies with device,Varies with device,1000000000,0
865,Google Play Games,ENTERTAINMENT,4.30000,7165362,Varies with device,Free,Teen,Entertainment,"July 16, 2018",Varies with device,Varies with device,1000000000,0
464,Hangouts,COMMUNICATION,4.00000,3419513,Varies with device,Free,Everyone,Communication,"July 21, 2018",Varies with device,Varies with device,1000000000,0
1917,Subway Surfers,GAME,4.50000,27725352,76M,Free,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up,1000000000,0
1700,Subway Surfers,GAME,4.50000,27723193,76M,Free,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up,1000000000,0
3565,Google Drive,PRODUCTIVITY,4.40000,2731211,Varies with device,Free,Everyone,Productivity,"August 6, 2018",Varies with device,Varies with device,1000000000,0


In [8]:
df = df.drop_duplicates(subset='App', keep='first', inplace=False)
df

,App,Category,Rating,Reviews,Size,Type,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Installs,Price
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.10000,159,19M,Free,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,10000,0
1,Coloring book moana,ART_AND_DESIGN,3.90000,967,14M,Free,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,500000,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.70000,87510,8.7M,Free,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,5000000,0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.50000,215644,25M,Free,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,50000000,0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.30000,967,2.8M,Free,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,100000,0
5,Paper flowers instructions,ART_AND_DESIGN,4.40000,167,5.6M,Free,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,50000,0
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.80000,178,19M,Free,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,50000,0
7,Infinite Painter,ART_AND_DESIGN,4.10000,36815,29M,Free,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up,1000000,0
8,Garden Coloring Book,ART_AND_DESIGN,4.40000,13791,33M,Free,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up,1000000,0
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.70000,121,3.1M,Free,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up,10000,0


In [9]:
df = df.drop('App', axis = 1)
df = df.drop('Category', axis = 1)
df = df.drop('Size', axis = 1)
df = df.drop('Type',axis = 1)
df = df.drop('Content Rating', axis = 1)
df = df.drop('Genres', axis = 1)
df = df.drop('Last Updated', axis = 1)
df = df.drop('Current Ver', axis = 1)
df = df.drop('Android Ver', axis = 1)
df.head()

,Rating,Reviews,Installs,Price
0,4.1,159,10000,0
1,3.9,967,500000,0
2,4.7,87510,5000000,0
3,4.5,215644,50000000,0
4,4.3,967,100000,0


In [10]:
mask = (df['Installs'] >= 1000000)
unmask = (df['Installs'] < 1000000)
df.loc[mask, 'Installs'] = 1
df.loc[unmask, 'Installs'] = 0

In [11]:
df['Installs'].value_counts()

0    6263
1    3395
Name: Installs, dtype: int64

In [12]:
df_train = df[:5420]
df_train.head()

,Rating,Reviews,Installs,Price
0,4.1,159,0,0
1,3.9,967,0,0
2,4.7,87510,1,0
3,4.5,215644,1,0
4,4.3,967,0,0


In [13]:
df_test = df[5420:]
df_test.head()

,Rating,Reviews,Installs,Price
6429,4.30000,15,0,3.99
6430,3.62319,0,0,0
6431,4.30000,855,0,0
6432,4.40000,18,0,2
6433,3.62319,4,0,0


In [14]:
y = df_train['Installs'].values
df_train = df_train.drop('Installs', 1)
y

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [15]:
dtree=DecisionTreeClassifier(max_depth=4,min_samples_split=10)
dtree.fit(df_train,y)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(df_train),
                class_names=['Unpopular','Popular'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree.pdf")

True

In [16]:
dtree.feature_importances_

array([0.01577033, 0.95873698, 0.0254927 ])

In [17]:
y_test = df_test['Installs'].values
X_test = df_test.drop('Installs', 1)

y_predict = dtree.predict(X_test)

y_predict

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [18]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

0.9646059462010382

![](https://i.imgur.com/wOLQV1u.png)

根據下載數的門檻設定不同，accuracy_score也會不同，測試過十萬、百萬、千萬到億，門檻設置的越高則accuracy_score也會越高。

以及資料的排序會影響預測的正確性，原本在分割tran及test dataset時，若已排序(根據下載數量)再做資料集的切割會使accuracy_score變高，
但有失準確性，故資料集要打亂再做分割，會比較好知道這顆決策樹真實的正確率。

這個說明是根據一百萬的決策樹下的結論，

當評論數大於8050且價錢小於1.24美元的時候，該App下載數大於一百萬的機率較高。

若價錢超過1.24美元時，評論數大於87242的App下載數會大於一百萬。

以及評分小於4.55但評論數超過6267時，也有機會下載數大於一百萬。